In [ ]:
#default_exp model.model_shop

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
import torch
import peptdeep.model.building_block as building_block
from peptdeep.model.model_interface import ModelInterface
from peptdeep.model.featurize import (
    get_ascii_indices, get_batch_mod_feature
)
import pandas as pd
import numpy as np

ASCII_NUM=128

# Regression models for predicting a scalar value for a given amino acid sequence

In [ ]:
#export

class ScalarRegression_LSTM_Model_for_AASeq(torch.nn.Module):
    def __init__(self, 
        *,
        hidden_dim=256,
        n_lstm_layers=4,
        dropout=0.1,
        **kwargs,
    ):
        super().__init__()
        self.dropout = torch.nn.Dropout(dropout)
        
        self.nn = torch.nn.Sequential(
            building_block.ascii_embedding(hidden_dim//4),
            building_block.SeqCNN(hidden_dim//4),
            self.dropout,
            building_block.SeqLSTM(
                hidden_dim, hidden_dim, 
                rnn_layer=n_lstm_layers
            ),
            building_block.SeqAttentionSum(hidden_dim),
            self.dropout,
            torch.nn.Linear(hidden_dim,64),
            torch.nn.GELU(),
            torch.nn.Linear(64, 1),
        )
    def forward(self, aa_x):
        return self.nn(aa_x).squeeze(-1)

class ScalarRegression_Transformer_Model_for_AASeq(torch.nn.Module):
    def __init__(self,
        *,
        hidden_dim = 256,
        nlayers = 4,
        output_attentions=False,
        dropout = 0.1,
        **kwargs,
    ):
        """
        Model based on a transformer Architecture from 
        Huggingface's BertEncoder class.
        """
        super().__init__()

        self.dropout = torch.nn.Dropout(dropout)

        self.input_nn =  torch.nn.Sequential(
            building_block.ascii_embedding(hidden_dim),
        )

        self.output_attentions = output_attentions
        
        self.hidden_nn = building_block.HFace_Transformer_with_PositionalEncoder(
            hidden_dim, nlayers=nlayers, dropout=dropout,
            output_attentions=output_attentions
        )

        self.output_nn = torch.nn.Sequential(
            building_block.SeqAttentionSum(hidden_dim),
            torch.nn.PReLU(),
            self.dropout,
            torch.nn.Linear(hidden_dim, 1),
        )

    @property
    def output_attentions(self)->bool:
        return self._output_attentions

    @output_attentions.setter
    def output_attentions(self, val:bool):
        self._output_attentions = val

    def forward(self, aa_x):
        aa_x = self.dropout(self.input_nn(aa_x))

        aa_x = self.hidden_nn(aa_x)
        if self.output_attentions:
            self.attentions = aa_x[1]
        else:
            self.attentions = None
        aa_x = self.dropout(aa_x[0])

        return self.output_nn(aa_x).squeeze(1)

class ScalarRegression_ModelInterface_for_AASeq(ModelInterface):
    def __init__(self, 
        dropout=0.1,
        model_class:torch.nn.Module=ScalarRegression_LSTM_Model_for_AASeq, #model defined above
        device:str='gpu',
        **kwargs,
    ):
        super().__init__(device=device)
        self.build(
            model_class,
            dropout=dropout,
            **kwargs
        )
        self.loss_func = torch.nn.L1Loss() # for regression

        self.target_column_to_predict = 'predicted_property'
        self.target_column_to_train = 'detected_property'

    def _prepare_predict_data_df(self,
        precursor_df:pd.DataFrame,
    ):
        self._predict_column_in_df = 'predicted_property'
        precursor_df[self._predict_column_in_df] = 0.
        self.predict_df = precursor_df

#### Building a RT model for only sequences based on `ScalarRegression_LSTM_Model_for_AASeq`

In [ ]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['detected_property'] = (
    IRT_PEPTIDE_DF.irt-IRT_PEPTIDE_DF.irt.min()
)/(IRT_PEPTIDE_DF.irt.max()-IRT_PEPTIDE_DF.irt.min())
model = ScalarRegression_ModelInterface_for_AASeq(
    model_class=ScalarRegression_LSTM_Model_for_AASeq
)
model.train(IRT_PEPTIDE_DF, epoch=20)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,detected_property,predicted_property
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.000000,0.000000
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,0.199488,0.186619
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,0.298671,0.292120
3,YILAGVENSK,RT-pep d,19.79,,,10,0.357909,0.381795
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,0.429315,0.405969
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,0.466699,0.515276
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0.537784,0.610525
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0.636728,0.691020
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0.764009,0.821196
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0.897775,0.942751


#### Building a RT model for only sequences based on `ScalarRegression_Transformer_Model_for_AASeq`

In [ ]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['detected_property'] = (
    IRT_PEPTIDE_DF.irt-IRT_PEPTIDE_DF.irt.min()
)/(IRT_PEPTIDE_DF.irt.max()-IRT_PEPTIDE_DF.irt.min())
model = ScalarRegression_ModelInterface_for_AASeq(
    model_class=ScalarRegression_Transformer_Model_for_AASeq
)
model.train(IRT_PEPTIDE_DF, epoch=20)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,detected_property,predicted_property
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.000000,0.102478
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,0.199488,0.262498
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,0.298671,0.261106
3,YILAGVENSK,RT-pep d,19.79,,,10,0.357909,0.283341
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,0.429315,0.397018
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,0.466699,0.483477
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0.537784,0.668417
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0.636728,0.559607
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0.764009,0.653198
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0.897775,0.842022


# Binary classification models for a given amino acid sequence

In [ ]:
#export
class BinaryClassification_LSTM_Model_for_AASeq(
    ScalarRegression_LSTM_Model_for_AASeq
):
    def __init__(self, 
        *,
        hidden_dim=256,
        n_lstm_layers=4,
        dropout=0.1,
        **kwargs,
    ):
        super().__init__(
            hidden_dim=hidden_dim,
            n_lstm_layers=n_lstm_layers,
            dropout=dropout,
        )

    def forward(self, aa_x):
        x = super().forward(aa_x)
        return torch.sigmoid(x)

class BinaryClassification_Transformer_Model_for_AASeq(
    ScalarRegression_Transformer_Model_for_AASeq
):
    def __init__(self,
        *,
        hidden_dim = 256,
        nlayers = 4,
        output_attentions=False,
        dropout = 0.1,
        **kwargs,
    ):
        """
        Model based on a transformer Architecture from 
        Huggingface's BertEncoder class.
        """
        super().__init__(
            nlayers=nlayers,
            hidden_dim=hidden_dim,
            output_attentions=output_attentions,
            dropout=dropout,
            **kwargs,
        )

    def forward(self, aa_x):
        x = super().forward(aa_x)
        return torch.sigmoid(x)

class BinaryClassification_ModelInterface_for_AASeq(ModelInterface):
    def __init__(self, 
        dropout=0.1,
        model_class:torch.nn.Module=BinaryClassification_LSTM_Model_for_AASeq, #model defined above
        device:str='gpu',
        **kwargs,
    ):
        """
        Class to predict retention times from precursor dataframes.
        """
        super().__init__(device=device)
        self.build(
            model_class,
            dropout=dropout,
            **kwargs
        )
        self.loss_func = torch.nn.BCELoss() # for binary classification
        self.target_column_to_predict = 'predicted_prob'
        self.target_column_to_train = 'detected_prob'

#### A sequence classification model using `BinaryClassification_LSTM_Model_for_AASeq`

In [ ]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['detected_prob'] = 0
IRT_PEPTIDE_DF.loc[:5,'detected_prob']=1
model = BinaryClassification_ModelInterface_for_AASeq(
    model_class=BinaryClassification_LSTM_Model_for_AASeq
)
model.train(IRT_PEPTIDE_DF, epoch=20)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,detected_prob,predicted_prob
0,LGGNEQVTR,RT-pep a,-24.92,,,9,1,0.994313
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,1,0.994523
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,1,0.994115
3,YILAGVENSK,RT-pep d,19.79,,,10,1,0.994055
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,1,0.992866
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,1,0.992847
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0,0.381737
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0,0.384261
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0,0.383973
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0,0.382710


#### A sequence classification model using `BinaryClassification_LSTM_Model_for_AASeq`

In [ ]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['detected_prob'] = 0
IRT_PEPTIDE_DF.loc[:5,'detected_prob']=1
model = BinaryClassification_ModelInterface_for_AASeq(
    model_class=BinaryClassification_Transformer_Model_for_AASeq
)
model.train(IRT_PEPTIDE_DF, epoch=10)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,detected_prob,predicted_prob
0,LGGNEQVTR,RT-pep a,-24.92,,,9,1,0.995645
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,1,0.993863
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,1,0.982984
3,YILAGVENSK,RT-pep d,19.79,,,10,1,0.994898
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,1,0.991012
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,1,0.988278
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0,0.016136
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0,0.036313
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0,0.015849
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0,0.013684


# Regression models for predicting a scalar value for a given amino acid sequence and site-specific PTMs

In [ ]:
#export
class ScalarRegression_LSTM_Model_for_ModAASeq(torch.nn.Module):
    def __init__(self, 
        *,
        hidden_dim=256,
        n_lstm_layers=4,
        dropout=0.1,
        **kwargs,
    ):
        super().__init__()
        self.dropout = torch.nn.Dropout(dropout)

        self.encoder_nn = building_block.Encoder_AsciiAA_Mod_CNN_LSTM_AttnSum(
            hidden_dim,
            n_lstm_layers=n_lstm_layers,
        )
        self.output_nn = torch.nn.Sequential(
            self.dropout,
            torch.nn.Linear(hidden_dim,64),
            torch.nn.GELU(),
            torch.nn.Linear(64, 1),
        )
    def forward(self, aa_x, mod_x):
        x = self.encoder_nn(aa_x, mod_x)
        return self.output_nn(x).squeeze(-1)

class ScalarRegression_Transformer_Model_for_ModAASeq(torch.nn.Module):
    def __init__(self,
        *,
        hidden_dim = 256,
        nlayers = 4,
        output_attentions=False,
        dropout = 0.1,
        **kwargs,
    ):
        """
        Model based on a transformer Architecture from 
        Huggingface's BertEncoder class.
        """
        super().__init__()

        self.dropout = torch.nn.Dropout(dropout)

        self.input_nn = building_block.AA_Mod_Embedding(hidden_dim)

        self._output_attentions = output_attentions
        
        self.hidden_nn = building_block.HFace_Transformer_with_PositionalEncoder(
            hidden_dim, nlayers=nlayers, dropout=dropout,
            output_attentions=output_attentions
        )

        self.output_nn = torch.nn.Sequential(
            building_block.SeqAttentionSum(hidden_dim),
            torch.nn.PReLU(),
            self.dropout,
            torch.nn.Linear(hidden_dim, 1),
        )

    @property
    def output_attentions(self)->bool:
        return self._output_attentions

    @output_attentions.setter
    def output_attentions(self, val:bool):
        self._output_attentions = val

    def forward(self, 
        aa_indices, 
        mod_x,
    ):
        x = self.dropout(self.input_nn(
            aa_indices, mod_x
        ))

        hidden_x = self.hidden_nn(x)
        if self.output_attentions:
            self.attentions = hidden_x[1]
        else:
            self.attentions = None
        x = self.dropout(hidden_x[0]+x*0.2)

        return self.output_nn(x).squeeze(1)

class ScalarRegression_ModelInterface_for_ModAASeq(ModelInterface):
    def __init__(self, 
        dropout=0.1,
        model_class:torch.nn.Module=ScalarRegression_LSTM_Model_for_ModAASeq, #model defined above
        device:str='gpu',
        **kwargs,
    ):
        super().__init__(device=device)
        self.build(
            model_class,
            dropout=dropout,
            **kwargs
        )
        self.loss_func = torch.nn.L1Loss() # for regression

        self.target_column_to_predict = 'predicted_property'
        self.target_column_to_train = 'detected_property'

    def _get_features_from_batch_df(self, 
        batch_df: pd.DataFrame,
        **kwargs,
    ):
        return (
            self._get_aa_indice_features(batch_df),
            self._get_mod_features(batch_df)
        )

#### Scalar regression model (RT) with modified AA sequences using `ScalarRegression_LSTM_Model_for_ModAASeq`

In [ ]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['detected_property'] = (
    IRT_PEPTIDE_DF.irt-IRT_PEPTIDE_DF.irt.min()
)/(IRT_PEPTIDE_DF.irt.max()-IRT_PEPTIDE_DF.irt.min())
model = ScalarRegression_ModelInterface_for_ModAASeq(
    model_class=ScalarRegression_LSTM_Model_for_ModAASeq
)
model.train(IRT_PEPTIDE_DF, epoch=20)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,detected_property,predicted_property
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.000000,0.000000
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,0.199488,0.380922
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,0.298671,0.350497
3,YILAGVENSK,RT-pep d,19.79,,,10,0.357909,0.399432
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,0.429315,0.500013
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,0.466699,0.579125
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0.537784,0.817981
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0.636728,0.854289
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0.764009,0.621173
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0.897775,0.872441


#### Scalar regression model (RT) with modified AA sequences using `ScalarRegression_Transformer_Model_for_ModAASeq`

In [ ]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['detected_property'] = (
    IRT_PEPTIDE_DF.irt-IRT_PEPTIDE_DF.irt.min()
)/(IRT_PEPTIDE_DF.irt.max()-IRT_PEPTIDE_DF.irt.min())
model = ScalarRegression_ModelInterface_for_ModAASeq(
    model_class=ScalarRegression_Transformer_Model_for_ModAASeq
)
model.train(IRT_PEPTIDE_DF, epoch=20)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,detected_property,predicted_property
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.000000,0.000000
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,0.199488,0.000000
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,0.298671,0.211310
3,YILAGVENSK,RT-pep d,19.79,,,10,0.357909,0.296768
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,0.429315,0.360999
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,0.466699,0.434928
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0.537784,0.350345
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0.636728,0.431385
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0.764009,0.722853
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0.897775,0.848814


# Binary Classification Models for Given Amino Acid Sequence and Site-specific PTMs

In [ ]:
#export
class BinaryClassification_LSTM_Model_for_ModAASeq(
    ScalarRegression_LSTM_Model_for_ModAASeq
):
    def __init__(self, 
        *,
        hidden_dim=256,
        n_lstm_layers=4,
        dropout=0.1,
        **kwargs,
    ):
        super().__init__(
            hidden_dim=hidden_dim,
            n_lstm_layers=n_lstm_layers,
            dropout=dropout,
            **kwargs,
        )

    def forward(self, aa_x, mod_x):
        x = super().forward(aa_x, mod_x)
        return torch.sigmoid(x)

class BinaryClassification_Transformer_Model_for_ModAASeq(
    ScalarRegression_Transformer_Model_for_ModAASeq
):
    def __init__(self,
        *,
        hidden_dim = 256,
        nlayers = 4,
        output_attentions=False,
        dropout = 0.1,
        **kwargs,
    ):
        """
        Model based on a transformer Architecture from 
        Huggingface's BertEncoder class.
        """
        super().__init__(
            nlayers=nlayers,
            hidden_dim=hidden_dim,
            output_attentions=output_attentions,
            dropout=dropout,
            **kwargs
        )

    @property
    def output_attentions(self)->bool:
        return self._output_attentions

    @output_attentions.setter
    def output_attentions(self, val:bool):
        self._output_attentions = val

    def forward(self, 
        aa_indices, 
        mod_x,
    ):
        x = super().forward(aa_indices, mod_x)
        return torch.sigmoid(x)

class BinaryClassification_ModelInterface_for_ModAASeq(ModelInterface):
    def __init__(self, 
        dropout=0.1,
        model_class:torch.nn.Module=BinaryClassification_LSTM_Model_for_ModAASeq, #model defined above
        device:str='gpu',
        **kwargs,
    ):
        super().__init__(device=device)
        self.build(
            model_class,
            dropout=dropout,
            **kwargs
        )
        self.loss_func = torch.nn.BCELoss() # for regression

        self.target_column_to_predict = 'predicted_prob'
        self.target_column_to_train = 'detected_prob'

    def _get_features_from_batch_df(self, 
        batch_df: pd.DataFrame,
    ):
        return (
            self._get_aa_indice_features(batch_df),
            self._get_mod_features(batch_df)
        )

#### Scalar regression model (RT) with modified AA sequences using `BinaryClassification_LSTM_Model_for_ModAASeq`

In [ ]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['detected_prob'] = 0
IRT_PEPTIDE_DF.loc[:5,'detected_prob']=1
model = BinaryClassification_ModelInterface_for_ModAASeq(
    model_class=BinaryClassification_LSTM_Model_for_ModAASeq
)
model.train(IRT_PEPTIDE_DF, epoch=20)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,detected_prob,predicted_prob
0,LGGNEQVTR,RT-pep a,-24.92,,,9,1,0.996101
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,1,0.995229
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,1,0.995115
3,YILAGVENSK,RT-pep d,19.79,,,10,1,0.995357
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,1,0.995352
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,1,0.991269
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0,0.394883
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0,0.398108
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0,0.399599
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0,0.390830


#### Scalar regression model (RT) with modified AA sequences using `BinaryClassification_Transformer_Model_for_ModAASeq`

In [ ]:
from peptdeep.model.rt import IRT_PEPTIDE_DF

IRT_PEPTIDE_DF=IRT_PEPTIDE_DF.copy()
IRT_PEPTIDE_DF['detected_prob'] = 0
IRT_PEPTIDE_DF.loc[:5,'detected_prob']=1
model = BinaryClassification_ModelInterface_for_ModAASeq(
    model_class=BinaryClassification_Transformer_Model_for_ModAASeq
)
model.train(IRT_PEPTIDE_DF, epoch=10)
model.predict(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,detected_prob,predicted_prob
0,LGGNEQVTR,RT-pep a,-24.92,,,9,1,0.995481
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,1,0.994566
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,1,0.982628
3,YILAGVENSK,RT-pep d,19.79,,,10,1,0.995643
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,1,0.991110
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,1,0.990920
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0,0.040117
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0,0.066093
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0,0.007390
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0,0.007112
